In [1]:
import torch

import av
print(av.__version__)
from torchvision import datasets, models, transforms
from torchvision.datasets import UCF101

8.0.3


In [2]:
ucf_data_dir = "/home/muhammadbsheikh/workspace/try/dataset/UCF-101"
ucf_label_dir = "/home/muhammadbsheikh/workspace/try/dataset/ucfTrainTestlist"
frames_per_clip = 5
step_between_clips = 1
batch_size = 32

In [3]:
tfs = transforms.Compose([
            # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
            # scale in [0, 1] of type float
            transforms.Lambda(lambda x: x / 255.),
            # reshape into (T, C, H, W) for easier convolutions
            transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
            # rescale to the most common size
            transforms.Lambda(lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [4]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

In [ ]:
# ucf101 data and loaders
train_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips, train=True, transform=tfs)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
# create test loader (allowing batches and other extras)
test_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                      step_between_clips=step_between_clips, train=False, transform=tfs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)

In [18]:
val_split = 0.05
total_train_samples = len(train_dataset)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(test_dataset)}")

number of train samples 1747931
number of validation samples 87397
number of test samples 682084


In [14]:
#!conda install -c conda-forge ipywidgets -y
#!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [15]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def update_acc(self, val, n=1):
        self.val = val/n
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count

In [16]:
def train(config, model, loader, optimizer, epoch):
    model.train()
    config = {}
    config['log_interval'] = 100
    correct = 0
    total_loss = 0.0
    flag = 0
    Loss, Acc = AverageMeter(), AverageMeter()
    start = time.time()
    for batch_id, data in enumerate(loader):
        data, target = data[0], data[-1]
        # print("here")

        if torch.cuda.is_available():
           data = data.cuda()
           target = target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        Loss.update(loss.item(), data.size(0))

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        num_corrects = pred.eq(target.view_as(pred)).sum().item()
        correct += num_corrects

        Acc.update_acc(num_corrects, data.size(0))

        if flag!= 0 and batch_id%config['log_interval'] == 0:
           print('Train Epoch: {} Batch [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}/{} ({:.0f})%'.format(
                epoch, batch_id * len(data), len(loader.dataset),
                100. * batch_id / len(loader), Loss.avg, correct, Acc.count, 100. * Acc.avg))
        flag = 1

    #total_loss /= len(loader.dataset) 
    print('Train Epoch: {} Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         epoch, Loss.avg, correct, Acc.count, 100. * Acc.avg ))
    print(f"Takes {time.time() - start}")

In [17]:
def test(config, model, loader, text='Validation'):
    model.eval()
    correct = 0
    total_loss = 0.0
    Loss, Acc = AverageMeter(), AverageMeter()
    with torch.no_grad():
         for batch_id, data in enumerate(loader):
             data, target = data[0], data[-1]

             if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

             output = model(data)
             loss = F.nll_loss(output, target)
             total_loss += loss.item()

             Loss.update(loss.item(), data.size(0))

             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
             num_corrects = pred.eq(target.view_as(pred)).sum().item()
             correct += num_corrects

             Acc.update_acc(num_corrects, data.size(0))
           
    total_loss /= len(loader.dataset)
    print(text + ' Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         Loss.avg, correct, Acc.count , 100. * Acc.avg ))

In [ ]:
from torch.utils.data import random_split,DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.nn import functional as F
import time as time
bs = 4
lr = 1e-2
gamma = 0.7
total_epochs = 10
config = {}
num_workers = 0

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}

hmdb51_train_v1, hmdb51_val_v1 = random_split(train_dataset, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

train_loader = DataLoader(hmdb51_train_v1, batch_size=bs, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=bs, shuffle=True, **kwargs)
test_loader  = DataLoader(test_dataset, batch_size=bs, shuffle=False, **kwargs)


lr = 0.0005
epochs = 1
gamma = 0.7


model = models.resnet101(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 101)
if torch.cuda.is_available():
    model= model.cuda()

optimizer = optim.Adam(network.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

print("Launching Action Recognition Model training")
for epoch in range(1, total_epochs + 1):
    train(config, model, train_loader, optimizer, epoch)
    test(config, model, val_loader, text="Validation")
    scheduler.step()

test(config, model, test_loader, text="Test")

In [2]:
cd ..

/home/muhammadbsheikh/workspace/try


In [15]:
# cnn2fft
# check PyTorch versions
# import standard PyTorch modules
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import torch
import torch.nn as nn
import torch.nn.functional as F
import pretrainedmodels
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# import torchvision module to handle image manipulation
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
from torch.autograd import Variable

# calculate train time, writing train data to files etc.
import time
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from IPython.display import clear_output


from efficientnet_pytorch import EfficientNet

from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import scipy
import scipy.fft



print(torch.__version__)
print(torchvision.__version__)

#device = torch.device("cuda")
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity
# torch.cuda.current_device()

# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json

# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

class RunManager():

  def __init__(self):

    # tracking every epoch count, loss, accuracy, time
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    
    self.epoch_val_loss = 0
    self.epoch_val_num_correct = 0
    self.epoch_start_time = None

    # tracking every run count, run data, hyper-params used, time
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    # record model, loader and TensorBoard 
    self.network = None
    self.loader = None
    self.vloader = None
    self.tb = None

  # record the count, hyper-param, model, loader of each run
  # record sample images and network graph to TensorBoard  
  def begin_run(self, run, network, loader,vloader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.vloader = vloader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    images, labels = images.to(device), labels.to(device)
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  # when run ends, close TensorBoard, zero epoch count
  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  # zero epoch count, loss, accuracy, 
  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0

  # 
  def end_epoch(self):
    # calculate epoch duration and run duration(accumulate)
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    # record epoch loss and accuracy
    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)
    
    vloss = self.epoch_val_loss / len(self.vloader.dataset)
    vaccuracy = self.epoch_val_num_correct / len(self.vloader.dataset)

    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
    
    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('vLoss', vloss, self.epoch_count)
    self.tb.add_scalar('vAccuracy', vaccuracy, self.epoch_count)

    # Record params to TensorBoard
    for name, param in self.network.named_parameters():
      self.tb.add_histogram(name, param, self.epoch_count)
      self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    
    # Write into 'results' (OrderedDict) for all run related data
    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["loss"] = loss
    results["val_loss"] = self.epoch_val_loss
    results["val_accuracy"] = self.epoch_val_num_correct
    results["accuracy"] = accuracy
    results["epoch duration"] = epoch_duration
    results["run duration"] = run_duration

    # Record hyper-params into 'results'
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)
    df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

    # display epoch information and show progress
    clear_output(wait=True)
    #display(df)

  def track_vloss_vacc(self, loss, acc):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_val_loss = loss.item()
    self.epoch_val_num_correct = acc
    
    
    # accumulate loss of batch into entire epoch loss
  def track_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_loss += loss.item() * self.loader.batch_size

  # accumulate number of corrects of batch into entire epoch num_correct
  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self._get_num_correct(preds, labels)

  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
  
  # save end results of all runs into csv, json for further a
  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, 
        orient = 'columns',
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
      json.dump(self.run_data, f, ensure_ascii=False, indent=4)

#set device
if torch.cuda.is_available():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_properties(device),torch.cuda.set_device(device),torch.cuda.current_device())

#Hyperparameters

#input_size
#num_classes = 50
#learning_rate = 0.00005
BATCH_SIZE = 32
epochs = 1

# put all hyper params into a OrderedDict, easily expandable
params = OrderedDict(
    exp='6',
    lr = [0.0005],#,0.001,0.0001,0.00001], # 0.001
    batch_size = [BATCH_SIZE], # 1000
    shuffle = [True] # True,False
    # Optimizer = [Adam,NAdam,RMSProp,Adamax,SGD,Adagrad,Adadelta]
)

TRAIN_DATA_PATH = "./datax/8/train/"
TEST_DATA_PATH = "./datax/8/test/"

transform = transforms.Compose([    
    transforms.Resize(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)



train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True,  num_workers=4)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True,  num_workers=4)

m = RunManager()


#prepare model
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

model.last_linear = nn.Identity() #freeze the model
#num_ftrs = model.last_linear.in_features
#model.last_linear = nn.Linear(num_ftrs, 50)

for p in model.parameters():
    p.requires_grad =False
    
#num_ftrs = model.last_linear.in_features

    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
#model.fc = nn.Linear(num_ftrs, 50)

PATH = "models/IRv2.pt"

torch.save(model,PATH)
#model = torch.load(PATH)

optimizer = optim.Adam(model.parameters(), lr=0.0005)

def train(model,loader,epochs=60):
    model.to(device)
    model.train()   
    print('Training...')
    for epoch in range(epochs):
        start = time.time()
        model.train()
        m.begin_epoch()
        running_loss=0

        for i,batch in enumerate(loader,0):
                images = batch[0]
                labels = batch[1]
                images = images.to(device)
                labels = labels.to(device)
                
                preds = model(images)
                loss = F.cross_entropy(preds, labels) # Adam, SGD, RSPROP

                optimizer.zero_grad()
                loss = Variable(loss, requires_grad = True)
                loss.backward()
                optimizer.step()

                running_loss+=loss.data

                if i%10==9:
                    end=time.time()
                    print ('[epoch %d,imgs %5d] loss: %.7f  time: %0.3f s'%(epoch+1,(i+1)*4,running_loss/100,(end-start)))
                    #tb.add_scalar('Loss', loss, epoch+1)
                    start=time.time()
                    running_loss=0    
    

train(model,train_data_loader)


def extract_features(model,dl):
    lbls = []
    model.eval()
    device = 'cuda:0'
    model.cuda(device)
    with torch.no_grad():
        features = None
        for batch in tqdm(dl, disable=True):
            
            images = batch[0]
            labels = batch[1]
            images = images.to(device)
            #labels = labels.to(device)

            output = model(images)
            lbls.append(labels)
            #print(labels)

            if features is not None:
                features = torch.cat((features, output), 0)

            else:
                features = output        
            

    return features.cpu().numpy(),lbls


def flatten_list(t):
    flat_list = [item for sublist in t for item in sublist]
    flat_list = np.array(flat_list)
    return flat_list



print('Extracting Features...')
feat,lbls = extract_features(model,train_data_loader)
# randomforest, logisticregression, SVM , KNN, LD,  


1.8.1
0.9.1
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28) None 0
Training...
[epoch 1,imgs    40] loss: 0.7387570  time: 1.370 s
[epoch 1,imgs    80] loss: 0.7379339  time: 0.906 s
[epoch 1,imgs   120] loss: 0.7349143  time: 0.908 s
[epoch 1,imgs   160] loss: 0.7383977  time: 0.908 s
[epoch 1,imgs   200] loss: 0.7410491  time: 0.919 s
[epoch 1,imgs   240] loss: 0.7397242  time: 0.910 s
[epoch 1,imgs   280] loss: 0.7403839  time: 0.907 s
[epoch 1,imgs   320] loss: 0.7420151  time: 0.907 s
[epoch 1,imgs   360] loss: 0.7408985  time: 0.904 s
[epoch 1,imgs   400] loss: 0.7445468  time: 0.905 s
[epoch 1,imgs   440] loss: 0.7357612  time: 0.907 s
[epoch 1,imgs   480] loss: 0.7383980  time: 0.907 s
[epoch 1,imgs   520] loss: 0.7396655  time: 0.908 s
[epoch 1,imgs   560] loss: 0.7339584  time: 0.917 s
[epoch 1,imgs   600] loss: 0.7349713  time: 0.907 s
[epoch 1,imgs   640] loss: 0.7398044  time: 0.911 s
[epoch 1,imgs   680] 

In [ ]:
#feat,lbls = extract_features(model,train_data_loader)
test_feat,test_lbls = extract_features(model,test_data_loader)
lbls = flatten_list(lbls)
test_lbls  =flatten_list(test_lbls) # flatting the lbls

In [ ]:
feat.shape,len(lbls),test_feat.shape,len(test_lbls)

((4835, 1536), 4835, (1854, 1536), 1854)

In [ ]:
from sklearn import metrics
print('Train-Without FFT')
#SVM
svm_preds = SVC(kernel='linear').fit(feat,lbls).predict(test_feat)
print("SVM Accuracy:",metrics.accuracy_score(test_lbls, svm_preds))


Train-Without FFT
SVM Accuracy: 0.4854368932038835


In [ ]:
#save np network features and labels
# Use pickle for serialization or np.save for saving np objects. two options



i = 0 #video index 4797
j = 0 # neuron index 1000

def hstft(S1):
    S21, S22 = np.split(S1,2)  # 500+500
    S311, S312 = np.split(S21,2) # 500    
    S321, S322 = np.split(S22,2) # 500
    #S1.shape,S21.shape,S22.shape,S311.shape,S312.shape,S321.shape,S322.shape    
    return np.concatenate((S1,S21,S22,S311,S312,S321,S322))

def alpha(features):
    A = np.zeros((len(features),4608))
    for i,f in enumerate(features):
        h = scipy.fft.fft(f)    
        #print(f.shape,h.shape)
        #print(h.shape)
        h = hstft(h)
        #print(h.shape)
        A[i,:] = h.real
    return A

train_alpha = alpha(feat)
test_alpha = alpha(test_feat)
train_alpha.shape,test_alpha.shape

((4835, 4608), (1854, 4608))

In [ ]:
X = train_alpha
Y = lbls
test_x = test_alpha
test_y = test_lbls
X.shape,Y.shape,test_x.shape,test_y.shape

((4835, 4608), (4835,), (1854, 4608), (1854,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#SVM
clf = SVC(kernel='linear').fit(X,Y)
preds = clf.predict(test_x)
print("SVM Accuracy:",metrics.accuracy_score(test_y, preds))

#KNN

knn_clf = KNeighborsClassifier(n_neighbors=3).fit(X,Y)
knn_preds = knn_clf.predict(test_x)
print("KNN Accuracy:",metrics.accuracy_score(test_y, knn_preds))

#Random Forest
rf_clf = RandomForestClassifier(n_estimators=100).fit(X,Y)
rf_preds = rf_clf.predict(test_x)
print("RF Accuracy:",metrics.accuracy_score(test_y, rf_preds))


SVM Accuracy: 0.45792880258899676
KNN Accuracy: 0.27993527508090615
RF Accuracy: 0.4039913700107875
